In [ ]:
import duckdb
from palmerpenguins import penguins
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

## Get Data


In [ ]:
con = duckdb.connect('my-db.duckdb')
df = penguins.load_penguins()
con.register('df',df)
con.execute('CREATE TABLE penguins AS SELECT * FROM df')
con.close()

In [ ]:
df = penguins.load_penguins().dropna()

df.head(3)

## Define Model and Fit


In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

model = LinearRegression().fit(X, y)

## Get some information


In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")

## turn our model into a Vetiver model and save it to /data/model


In [ ]:
from vetiver import VetiverModel
v = VetiverModel(model, model_name='penguin_model', prototype_data=X)

In [ ]:
from vetiver import vetiver_pin_write
from vetiver import VetiverAPI
import pins

In [ ]:
b = pins.board_folder('data/model', allow_pickle_read=True)
vetiver_pin_write(b,v,'penguin_model')
v = VetiverModel.from_pin(b, 'penguin_model')

## Generate a {fastAPI} model


In [ ]:
app = VetiverAPI(v,check_prototype = True)

## try calling the model in the code

In [ ]:
import requests

#app.run(port = 8080)  # run the app

req_data = {
  "bill_length_mm": 0,
  "species_Chinstrap": False,
  "species_Gentoo": False,
  "sex_male": False
}
req = requests.post('http://127.0.0.1:8080/predict', json = [req_data])
res = req.json().get('predict')[0]

# run this to open a python virtual environment: .venv\Scripts\activate